In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import shuffle
from music21 import *
from glob import glob
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, Dropout
from keras.utils import np_utils

%run lstm_model.ipynb
%run data_preprocess.ipynb
%run generate_music.ipynb
np.set_printoptions(threshold=1000000)
bach_chorale_path = 'datasets/bach2'
parts_index = [0,1,2,3]
#parts_index = [0]
note_type = set()
"""
note_type:
0-127 : note pitch
128: slur-1st-part
129: slur-2nd-part
130: slur-3rd-part
131: slur-4th-part
132: rest
133: start symbol
134: end symbol
"""

def filter_files(files):
    file_list = []
    file_count = 0
    for i, file in enumerate(files, 1):
        s = converter.parse(file)
        if len(s.parts) == len(parts_index):
            print(i, file, len(s.parts))
            file_count += 1
            file_list.append(file)
    return file_list, file_count            


def make_note_type_list():
    note_type_list = []
    for i in note_type:
        note_type_list.append(i)
    return sorted(note_type_list)
# def transform_note_to_index(notes):
#     for time in range(notes.shape[0]):
#         for p_index in range(4):
#             notes[time][p_index] = note_dict[notes[time][p_index]]
#     return notes
def index_note_transform(inputs, option, pn):
    transformed_inputs = []
    if option == "to_note":
        note_dict = dict((index, note) for index, note in enumerate(note_type))
    elif option == "to_index":
        note_dict = dict((note, index) for index, note in enumerate(note_type))
        
    for p_index in range(pn):
        transformed_part = []
        for n in range(inputs.shape[1]):
            for keys in note_dict:
                if option == "to_note":
                    if inputs[p_index][n] == keys:
                        transformed_part.append(note_dict[keys])
                elif option == "to_index":
                    if frozenset(inputs[p_index][n]) == keys:
                        transformed_part.append(note_dict[keys])
        transformed_inputs.append(transformed_part)
    return np.array(transformed_inputs)
def notes_to_piano_roll(dataset_notes):
#     start_symbol = np.full((16, 1),133)
#     end_symbol = np.full((16, 1),134)
    note_type.add(frozenset([133]))
    note_type.add(frozenset([134]))
    for chorale_index in range(len(dataset_notes)):
        chorale = np.array(dataset_notes[chorale_index])
#         for i in range(len(parts_index)):
#             piano_roll_notes.append(np.concatenate((start_symbol, chorale[i], end_symbol)))
        chorale = index_note_transform(chorale, "to_index", len(parts_index))
        piano_roll_notes = np.transpose(chorale)
        #piano_roll_notes = index_note_transform(transposed_chorale, "to_index", 4)
        yield piano_roll_notes
def notes_to_onehot(notes):
    note_dict = note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        a = []
        for note in n:
            a.append(np.array(note == np.arange(0, len(note_dict)), dtype=np.int32))
        onehot.append(a)
    return np.array(onehot)
def part_notes_to_onehot(notes):
    note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        a = np.zeros(len(note_dict,),dtype=np.int32)
        for note in n:
            a[note] = 1
        onehot.append(a)
    return np.array(onehot)

def load_files():
    if os.path.exists(bach_chorale_path):
#         file_list = filter_files(glob(bach_chorale_path))
        file_list, num_file = filter_files(glob(bach_chorale_path + '/*.mxl') +
                                 glob(bach_chorale_path + '/*.mid'))
        print(num_file)
        dataset_notes = make_dataset(file_list)
        shuffle(dataset_notes)
        chorale_list = [notes for chorale_index, notes in enumerate(notes_to_piano_roll(dataset_notes))]
        X_train = chorale_list[0]
        X_test = chorale_list[int(len(chorale_list) * 0.8)]
        notes = chorale_list[0]
        
        for i, n in enumerate(chorale_list):
            if i != 0:
                notes = np.concatenate((notes, n))
                if i < int(len(chorale_list) * 0.8):
                    X_train = np.concatenate((X_train, n))
                elif i > int(len(chorale_list) * 0.8):
                    X_test = np.concatenate((X_test, n))
        print(notes.shape)
    
        X_train_onehot_notes = part_notes_to_onehot(X_train)
        X_test_onehot_notes = part_notes_to_onehot(X_test)
        Y_train_onehot_notes = notes_to_onehot(X_train)
        Y_test_onehot_notes = notes_to_onehot(X_test)
        for i in range(len(parts_index)):
            lstm_model = LSTM_Model(X=X_train_onehot_notes,Y=Y_train_onehot_notes,
                                    num_units=75, epochs=7, output_dim=len(note_type), name="bach"+str(i))
            gen = ((X,Y) for (X, Y) in lstm_model.generator(X_train_onehot_notes, Y_train_onehot_notes, i))
            test_gen = ((X,Y) for (X,Y) in lstm_model.generator(X_test_onehot_notes, Y_test_onehot_notes, i))
            lstm_model.create_model()
            lstm_model.train_model(gen, test_gen)

        score = generate_notes(lstm_model, notes)
                
        output_file = str(lstm_model.epochs) + '-epochs_' + str(num_file) + '-samples.mid'
        mf = midi.translate.music21ObjectToMidiFile(score)
        mf.open(output_file, 'wb')
        mf.write()
        mf.close()
        print("File " + output_file + " written")
    else:
        print("The file path is wrong!!")
    
def main():
    load_files()
if __name__ == "__main__":
    main()

1 datasets/bach2\bach_302.mid 4
2 datasets/bach2\bach_303.mid 4
3 datasets/bach2\bach_304.mid 4
4 datasets/bach2\bach_305.mid 4
5 datasets/bach2\bach_306.mid 4
6 datasets/bach2\bach_307.mid 4
7 datasets/bach2\bach_308.mid 4
8 datasets/bach2\bach_309.mid 4
9 datasets/bach2\bach_310.mid 4
10 datasets/bach2\bach_311.mid 4
10


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.02it/s]


(2640, 4)
create model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 16, 75)            36900     
_________________________________________________________________
dropout_21 (Dropout)         (None, 16, 75)            0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 16, 75)            45300     
_________________________________________________________________
dropout_22 (Dropout)         (None, 16, 75)            0         
_________________________________________________________________
dense_11 (Dense)             (None, 16, 47)            3572      
Total params: 85,772
Trainable params: 85,772
Non-trainable params: 0
_________________________________________________________________
train model...
Epoch 1/7
256/256 [==============================] - ETA: 1140s - loss: 3.85 - ETA: 383s - loss: 

256/256 [==============================] - ETA: 9s - loss: 0.883 - ETA: 8s - loss: 1.171 - ETA: 8s - loss: 0.988 - ETA: 8s - loss: 0.980 - ETA: 8s - loss: 0.932 - ETA: 8s - loss: 0.907 - ETA: 8s - loss: 0.873 - ETA: 7s - loss: 0.862 - ETA: 8s - loss: 0.841 - ETA: 8s - loss: 0.850 - ETA: 7s - loss: 0.851 - ETA: 7s - loss: 0.859 - ETA: 7s - loss: 0.857 - ETA: 7s - loss: 0.889 - ETA: 7s - loss: 0.898 - ETA: 7s - loss: 0.906 - ETA: 7s - loss: 0.904 - ETA: 7s - loss: 0.908 - ETA: 7s - loss: 0.912 - ETA: 7s - loss: 0.911 - ETA: 7s - loss: 0.905 - ETA: 7s - loss: 0.900 - ETA: 7s - loss: 0.897 - ETA: 7s - loss: 0.893 - ETA: 7s - loss: 0.887 - ETA: 7s - loss: 0.883 - ETA: 7s - loss: 0.882 - ETA: 7s - loss: 0.884 - ETA: 7s - loss: 0.877 - ETA: 7s - loss: 0.871 - ETA: 7s - loss: 0.864 - ETA: 7s - loss: 0.858 - ETA: 6s - loss: 0.849 - ETA: 6s - loss: 0.846 - ETA: 6s - loss: 0.845 - ETA: 6s - loss: 0.843 - ETA: 7s - loss: 0.840 - ETA: 7s - loss: 0.837 - ETA: 7s - loss: 0.831 - ETA: 6s - loss: 0.826

256/256 [==============================] - ETA: 9s - loss: 0.503 - ETA: 9s - loss: 0.482 - ETA: 8s - loss: 0.519 - ETA: 8s - loss: 0.543 - ETA: 8s - loss: 0.561 - ETA: 8s - loss: 0.576 - ETA: 8s - loss: 0.558 - ETA: 8s - loss: 0.590 - ETA: 8s - loss: 0.593 - ETA: 8s - loss: 0.633 - ETA: 8s - loss: 0.618 - ETA: 8s - loss: 0.625 - ETA: 8s - loss: 0.630 - ETA: 8s - loss: 0.620 - ETA: 8s - loss: 0.619 - ETA: 8s - loss: 0.609 - ETA: 8s - loss: 0.611 - ETA: 8s - loss: 0.612 - ETA: 8s - loss: 0.611 - ETA: 8s - loss: 0.613 - ETA: 7s - loss: 0.620 - ETA: 7s - loss: 0.624 - ETA: 7s - loss: 0.620 - ETA: 7s - loss: 0.619 - ETA: 7s - loss: 0.618 - ETA: 7s - loss: 0.622 - ETA: 7s - loss: 0.625 - ETA: 7s - loss: 0.632 - ETA: 7s - loss: 0.636 - ETA: 7s - loss: 0.636 - ETA: 7s - loss: 0.638 - ETA: 7s - loss: 0.639 - ETA: 7s - loss: 0.641 - ETA: 7s - loss: 0.641 - ETA: 6s - loss: 0.638 - ETA: 6s - loss: 0.638 - ETA: 6s - loss: 0.630 - ETA: 6s - loss: 0.633 - ETA: 6s - loss: 0.630 - ETA: 6s - loss: 0.627

...

256/256 [==============================] - ETA: 1396s - loss: 3.88 - ETA: 702s - loss: 3.8752 - ETA: 470s - loss: 3.854 - ETA: 354s - loss: 3.842 - ETA: 285s - loss: 3.817 - ETA: 239s - loss: 3.800 - ETA: 206s - loss: 3.773 - ETA: 181s - loss: 3.748 - ETA: 162s - loss: 3.705 - ETA: 146s - loss: 3.691 - ETA: 133s - loss: 3.645 - ETA: 123s - loss: 3.602 - ETA: 106s - loss: 3.497 - ETA: 99s - loss: 3.433 - ETA: 94s - loss: 3.39 - ETA: 89s - loss: 3.32 - ETA: 84s - loss: 3.35 - ETA: 80s - loss: 3.37 - ETA: 76s - loss: 3.31 - ETA: 73s - loss: 3.25 - ETA: 70s - loss: 3.22 - ETA: 67s - loss: 3.16 - ETA: 65s - loss: 3.15 - ETA: 62s - loss: 3.11 - ETA: 60s - loss: 3.08 - ETA: 58s - loss: 3.03 - ETA: 56s - loss: 3.01 - ETA: 55s - loss: 2.96 - ETA: 53s - loss: 2.96 - ETA: 50s - loss: 2.91 - ETA: 47s - loss: 2.88 - ETA: 44s - loss: 2.83 - ETA: 42s - loss: 2.78 - ETA: 40s - loss: 2.74 - ETA: 38s - loss: 2.70 - ETA: 36s - loss: 2.71 - ETA: 35s - loss: 2.74 - ETA: 33s - loss: 2.69 - ETA: 32s - loss: 

256/256 [==============================] - ETA: 8s - loss: 0.531 - ETA: 9s - loss: 0.600 - ETA: 9s - loss: 0.648 - ETA: 8s - loss: 0.747 - ETA: 8s - loss: 0.766 - ETA: 8s - loss: 0.812 - ETA: 8s - loss: 0.796 - ETA: 8s - loss: 0.813 - ETA: 8s - loss: 0.780 - ETA: 8s - loss: 0.803 - ETA: 8s - loss: 0.799 - ETA: 8s - loss: 0.815 - ETA: 8s - loss: 0.804 - ETA: 8s - loss: 0.866 - ETA: 8s - loss: 0.848 - ETA: 8s - loss: 0.839 - ETA: 8s - loss: 0.834 - ETA: 8s - loss: 0.822 - ETA: 8s - loss: 0.824 - ETA: 8s - loss: 0.826 - ETA: 8s - loss: 0.840 - ETA: 7s - loss: 0.854 - ETA: 7s - loss: 0.854 - ETA: 7s - loss: 0.859 - ETA: 7s - loss: 0.873 - ETA: 7s - loss: 0.882 - ETA: 7s - loss: 0.880 - ETA: 7s - loss: 0.876 - ETA: 7s - loss: 0.878 - ETA: 7s - loss: 0.880 - ETA: 7s - loss: 0.879 - ETA: 7s - loss: 0.881 - ETA: 7s - loss: 0.875 - ETA: 7s - loss: 0.879 - ETA: 6s - loss: 0.878 - ETA: 6s - loss: 0.880 - ETA: 6s - loss: 0.880 - ETA: 6s - loss: 0.869 - ETA: 6s - loss: 0.870 - ETA: 6s - loss: 0.869

256/256 [==============================] - ETA: 10s - loss: 1.01 - ETA: 9s - loss: 0.7194 - ETA: 9s - loss: 0.652 - ETA: 9s - loss: 0.642 - ETA: 9s - loss: 0.610 - ETA: 9s - loss: 0.597 - ETA: 9s - loss: 0.588 - ETA: 9s - loss: 0.595 - ETA: 8s - loss: 0.581 - ETA: 9s - loss: 0.573 - ETA: 8s - loss: 0.546 - ETA: 8s - loss: 0.551 - ETA: 8s - loss: 0.554 - ETA: 8s - loss: 0.578 - ETA: 8s - loss: 0.573 - ETA: 8s - loss: 0.574 - ETA: 8s - loss: 0.555 - ETA: 8s - loss: 0.576 - ETA: 8s - loss: 0.569 - ETA: 8s - loss: 0.584 - ETA: 8s - loss: 0.579 - ETA: 8s - loss: 0.615 - ETA: 8s - loss: 0.606 - ETA: 8s - loss: 0.609 - ETA: 8s - loss: 0.603 - ETA: 8s - loss: 0.594 - ETA: 7s - loss: 0.592 - ETA: 7s - loss: 0.593 - ETA: 7s - loss: 0.595 - ETA: 7s - loss: 0.604 - ETA: 7s - loss: 0.605 - ETA: 7s - loss: 0.605 - ETA: 7s - loss: 0.613 - ETA: 7s - loss: 0.618 - ETA: 7s - loss: 0.614 - ETA: 7s - loss: 0.613 - ETA: 7s - loss: 0.618 - ETA: 7s - loss: 0.619 - ETA: 7s - loss: 0.618 - ETA: 7s - loss: 0.61

256/256 [==============================] - ETA: 11s - loss: 0.13 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.14 - ETA: 11s - loss: 0.24 - ETA: 11s - loss: 0.24 - ETA: 10s - loss: 0.26 - ETA: 10s - loss: 0.26 - ETA: 10s - loss: 0.26 - ETA: 10s - loss: 0.27 - ETA: 10s - loss: 0.29 - ETA: 9s - loss: 0.2797 - ETA: 9s - loss: 0.275 - ETA: 9s - loss: 0.273 - ETA: 9s - loss: 0.280 - ETA: 9s - loss: 0.283 - ETA: 9s - loss: 0.276 - ETA: 9s - loss: 0.280 - ETA: 9s - loss: 0.285 - ETA: 9s - loss: 0.282 - ETA: 9s - loss: 0.283 - ETA: 9s - loss: 0.276 - ETA: 9s - loss: 0.273 - ETA: 8s - loss: 0.270 - ETA: 8s - loss: 0.281 - ETA: 8s - loss: 0.287 - ETA: 8s - loss: 0.289 - ETA: 8s - loss: 0.287 - ETA: 8s - loss: 0.284 - ETA: 8s - loss: 0.292 - ETA: 8s - loss: 0.301 - ETA: 8s - loss: 0.298 - ETA: 8s - loss: 0.305 - ETA: 8s - loss: 0.305 - ETA: 8s - loss: 0.323 - ETA: 8s - loss: 0.323 - ETA: 8s - loss: 0.320 - ETA: 8s - loss: 0.322 - ETA: 8s - loss: 0.319 - ETA: 8s - loss: 0.317 - ETA: 8s - loss: 0.31

  frozenset({134}) frozenset({134}) frozenset({134}) frozenset({134})]]
File 7-epochs_10-samples.mid written


In [ ]:
n = [0,1,2,3,4,5,6,7,8]
n = np.array(n)
g = np.zeros((len(n),),dtype=np.int32)
print(g)
a = [0,5,7]
for i in a:
    print(np.array(i == np.arange(0, len(n) )))

In [86]:
def inner_pro(x1,x2,y1,y2):
    return (x1*y1)+(x2*y2)
x = [1.4, 1.6]
A1 = [1.5, 2.0, 1.6, 1.2, 1.5]
A2 = [1.7, 1.9, 1.8, 1.5, 1.0]
ans = []
for i in range(5):
    ans.append(inner_pro(x[0],x[1],A1[i],A2[i]) / (inner_pro(x[0],x[1],x[0],x[1]) * inner_pro(A1[i],A2[i],A1[i],A2[i])))
ans

[0.2074653076684687,
 0.16978126126545184,
 0.19530057979859625,
 0.2446219152457011,
 0.2518720217835262]

In [68]:
f = frozenset([1,2,3,4,5])
for i in f:
    print(i)

5
